## @staticmethod and @classmethod

In the previous section we saw how to use function overloading in Python using @singledispatch. When you overload a method you use classmethod instead. Both @staticmethod and @classmethod don't have first paramter as self

### @staticmethods 
@staticmethods are like plain normal functions, just that they are contained in classes. 

### @classmethods
@classmethods are functions where the first argument passed in the class itself. A common use of this method is to overload constructors

In [6]:
class Car:
    def __init__(self, car_name, max_speed):
        self.car_name = car_name
        self.max_speed = max_speed
    
    @classmethod
    def from_code(cls, car_code):
        car_name, max_speed = car_code.split("-")
        car_inst = cls(car_name, max_speed) # using cls to instantiate the object
        return car_inst
    
    @staticmethod
    def print_me():
        print("I am a car")

volkswagen = Car.from_code("Jetta-200")
print(volkswagen.car_name, volkswagen.max_speed)

lamborghini = Car("Lambo", "450")
print(lamborghini.car_name, lamborghini.max_speed)

Jetta 200
Lambo 450


You can also use @staticmethod to overload the class, but @staticmethod is preferred for following reasons

In [10]:
class Car:
    """Car class overloaded with staticmethod"""
    def __init__(self, car_name, max_speed):
        self.car_name = car_name
        self.max_speed = max_speed
    
    @staticmethod
    def from_code(car_code):
        return Car(*car_code.split("-"))

class FordCar(Car):
    def __init__(self, ):
        pass

lamborghini = Car.from_code("Lamborghini-450")
mustang = FordCar.from_code("MustangGT-400")
print(isinstance(lamborghini, Car))
print(isinstance(mustang, Car))
print(isinstance(mustang, FordCar)) # Mustang is not an instance of Ford car! Which is undesirable!

True
True
False


Since we have hardcoded the from_code method to return a car instance, the subclases that inherit from that class are not instance of themselves! To fix this, use @staticmethod

In [4]:
class Car:
    """Car class overloaded with classmethod"""
    def __init__(self, car_name, max_speed):
        self.car_name = car_name
        self.max_speed = max_speed
    
    @classmethod
    def from_code(cls, car_code):
        return cls(*car_code.split("-"))

class FordCar(Car):
    def __init__(self, car_name, max_speed):
        self.car_name = car_name
        self.max_speed = max_speed
        
lamborghini = Car.from_code("Lamborghini-450")
mustang = FordCar.from_code("MustangGT-400")
print(isinstance(lamborghini, Car))
print(isinstance(mustang, Car))
print(isinstance(mustang, FordCar)) # Now the output is correct!

True
True
True


## Private variables, and function 
If you prefix your variables, or function with double underscore, then Python will mangle the name of that variable thus making it (sort of) private. This feature is called "name mangling". It is a prevention mechanism for not overriding the variable names accidentally, and not a __security measure__. If you're bent on chaning the variables then you can do that. 

In [20]:
class Person:
    def __init__(self, name, SSN):
        self.name = name
        self.__ssn = SSN
    def __secret_method(self):
        print("I am secret and invisible")
Mayur = Person("Mayur", 123435345)
print(Mayur.name)
print(Mayur.__ssn) # will throw error
# or this Mayur.__secret_method() will throw error too!

Mayur


AttributeError: 'Person' object has no attribute '__ssn'

In [21]:
Person.__dict__ # however, the secrect method is visible in __dict__ 

mappingproxy({'_Person__secret_method': <function __main__.Person.__secret_method>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__doc__': None,
              '__init__': <function __main__.Person.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Person' objects>})

In [22]:
Mayur._Person__secret_method() # executes just fine!

I am secret and invisible


## Property and making variables read-only
@property is a way of building descriptor that triggers function calls based on the function calls to that attribute. Property has following signature. 

```property(fget=None, fset=None, fdel=None, doc=None) -> property attribute```

You can set your custom getters, setters, deleters, and doc string to it


In [12]:
class Car:
    def __init__(self, car_name, max_speed):
        self.__car_name = car_name
        self.max_speed = max_speed
    
    @property
    def car_name(self):
        return self.__car_name
    
    @car_name.setter
    def car_name(self, new_car_name):
        """Setter to the car name private attribute"""
        self.__car_name = new_car_name
        
    @car_name.deleter
    def car_name(self, new_car_name):
        """Deleter to the car name private attribute"""
        del self.__car_name

Innova = Car("Innova", 250)
print(Innova.car_name)
Innova.car_name = "BMW"
print(Innova.car_name)

Innova
BMW


### Making Read only objects
Sometimes you want some fields immutable (For instance, the car name is above example) When you use property and don't implement the setter method then the object value becomes immutable.  

In [14]:
class Car:
    def __init__(self, car_name, max_speed):
        self.__car_name = car_name
        self.max_speed = max_speed
    
    @property
    def car_name(self):
        return self.__car_name
    
#     Don't implement setter  
#     @car_name.setter
#     def car_name(self, new_car_name):
#         """Setter to the car name private attribute"""
#         self.__car_name = new_car_name
        
    @car_name.deleter
    def car_name(self, new_car_name):
        """Deleter to the car name private attribute"""
        del self.__car_name

Innova = Car("Innova", 250)
print(Innova.car_name)
Innova.car_name = "BMW" # cannot set attribute!
print(Innova.car_name)

Innova


AttributeError: can't set attribute

## Saving memory and \__slots\__
Python stores all the class attributes in \__dict\__ so you can dynamically add in stuff even after the function is defined

In [27]:
sedan = Car("sedan", 123)
print(sedan.car_name)
sedan.wheels = "MRF" # you can add anything
print(sedan.wheels)

sedan
MRF


In [29]:
sedan.__dict__ # This will show the stuff inside this instance variable

{'_Car__car_name': 'sedan', 'max_speed': 123, 'wheels': 'MRF'}